Step 1: Install Necessary Libraries

In [4]:
# Install necessary libraries
!pip install groq pypdf


Step 2: Import Libraries

In [5]:
# Importing necessary libraries
from pypdf import PdfReader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq
import json


Step 3: Function to Extract Text from PDF

In [6]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.

    """
    reader = PdfReader(pdf_path)
    resume_text = ""

    for i in range(len(reader.pages)):
        page = reader.pages[i]
        resume_text += page.extract_text()

    return resume_text


Step 4: Function to Extract Information Using Groq API

In [7]:
def extract_resume_info(api_key, resume_text):
    """
    Extracts information from the resume text using Groq API.

    """
    client = Groq(api_key=api_key)
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "user",
                "content": f"""
                I have a resume text. Extract the following information:

                - Projects (project short summary, duration)
                - Experience (Company, Role, Duration) include internships
                - Skills(only Skills) remove categories

                Resume:
                {resume_text}

                Please provide the extracted information in a structured JSON format.
                """
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    response = ""
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""

    start_index = response.find('{')
    end_index = response.rfind('}') + 1
    json_str = response[start_index:end_index]
    json_data = json.loads(json_str)

    return json_data


Step 5: Function to Calculate Similarity Between Resume and Job Descriptions

In [8]:
def calculate_similarity(resume_text, job_description):
    """
    Calculates the similarity between resume text and a job description.

    """
    rtext = [resume_text, job_description]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(rtext)
    match = cosine_similarity(count_matrix)[0][1] * 100
    return round(match, 2)


Step 6: Function to Recommend Jobs Based on Resume

In [9]:
def recommend_jobs(resume_text, jobs):
    """
    Recommends jobs based on the similarity between the resume and job descriptions.

    """
    matches = []
    for job_des in jobs:
        match = calculate_similarity(resume_text, job_des['job_description'])
        matches.append({"job_name": job_des['job_name'], "match": match})

    sorted_matches = sorted(matches, key=lambda x: x['match'], reverse=True)
    return sorted_matches


Step 7: Main Function to Run the Notebook

In [10]:
def main():
    # Step 1: Extract text from PDF
    pdf_path = input("Enter Path to resume: ")
    resume_text = extract_text_from_pdf(pdf_path)
    print(f"Resume Text:\n{resume_text}\n")

    # Step 2: Extract structured information from resume using Groq API
    api_key = "" #API key is shared for security concern
    extracted_info = extract_resume_info(api_key, resume_text)
    print(f"Extracted Resume Information:\n{json.dumps(extracted_info, indent=2)}\n")

    # Step 3: Job descriptions
    jobs = [
    {
        "job_name": "Software Engineer",
        "job_description": "Responsible for designing, developing, and maintaining software applications. Required skills: Proficiency in programming languages like Java, C#, or Python, experience with version control systems (e.g., Git), understanding of software development methodologies (e.g., Agile), and strong problem-solving abilities."
    },
    {
        "job_name": "Data Scientist",
        "job_description": "Analyze and interpret complex data to help companies make better decisions. Required skills: Proficiency in Python or R, experience with machine learning algorithms, knowledge of data visualization tools (e.g., Tableau, Power BI), and strong statistical analysis skills."
    },
    {
        "job_name": "DevOps Engineer",
        "job_description": "Facilitate collaboration between development and operations teams to improve deployment and automation processes. Required skills: Experience with CI/CD tools (e.g., Jenkins, GitLab CI), proficiency in scripting languages (e.g., Bash, Python), knowledge of cloud platforms (e.g., AWS, Azure), and understanding of containerization (e.g., Docker, Kubernetes)."
    },
    {
        "job_name": "Cybersecurity Analyst",
        "job_description": "Protect an organization's IT infrastructure by identifying vulnerabilities and implementing security measures. Required skills: Knowledge of security protocols and standards (e.g., ISO 27001), experience with security tools (e.g., firewalls, IDS/IPS), proficiency in network security, and strong analytical and problem-solving abilities."
    },
    {
        "job_name": "Front-End Developer",
        "job_description": "Develop and maintain user-facing features of web applications. Required skills: Proficiency in HTML, CSS, and JavaScript, experience with front-end frameworks (e.g., React, Angular, Vue.js), understanding of responsive design principles, and ability to work with RESTful APIs."
    },
    {
        "job_name": "Back-End Developer",
        "job_description": "Develop and maintain the server-side logic of web applications. Required skills: Proficiency in server-side languages (e.g., Node.js, Python, Java), experience with databases (e.g., MySQL, MongoDB), knowledge of API design and development, and understanding of security best practices."
    },
    {
        "job_name": "Cloud Architect",
        "job_description": "Design and oversee the deployment of cloud-based solutions for an organization. Required skills: Experience with cloud platforms (e.g., AWS, Azure, Google Cloud), knowledge of cloud architecture best practices, proficiency in cloud automation tools (e.g., Terraform, CloudFormation), and strong problem-solving skills."
    },
    {
        "job_name": "IT Support Specialist",
        "job_description": "Provide technical assistance to end-users and maintain IT systems. Required skills: Proficiency in troubleshooting hardware and software issues, knowledge of operating systems (e.g., Windows, Linux, macOS), experience with help desk software (e.g., ServiceNow, Jira), and strong communication skills."
    },
    {
        "job_name": "Database Administrator",
        "job_description": "Manage and maintain an organization's databases, ensuring data integrity and performance. Required skills: Proficiency in SQL and database management systems (e.g., Oracle, MySQL, PostgreSQL), experience with backup and recovery procedures, knowledge of database security practices, and strong analytical skills."
    },
    {
        "job_name": "AI/ML Engineer",
        "job_description": "Design and develop machine learning models and AI-based solutions. Required skills: Proficiency in Python, experience with machine learning frameworks (e.g., TensorFlow, PyTorch), knowledge of data preprocessing techniques, and understanding of algorithms and data structures."
    },
    {
        "job_name": "Network Engineer",
        "job_description": "Design, implement, and manage the network infrastructure for an organization. Required skills: Proficiency in networking protocols (e.g., TCP/IP, DNS, BGP), experience with network hardware (e.g., routers, switches, firewalls), knowledge of network security practices, and strong troubleshooting abilities."
    },
    {
        "job_name": "QA Engineer",
        "job_description": "Ensure the quality and reliability of software applications through rigorous testing. Required skills: Experience with software testing methodologies (e.g., unit testing, integration testing, regression testing), proficiency in test automation tools (e.g., Selenium, JUnit), knowledge of bug tracking tools (e.g., Jira), and strong attention to detail."
    },
    {
        "job_name": "UI/UX Designer",
        "job_description": "Design user interfaces and experiences for web and mobile applications. Required skills: Proficiency in design tools (e.g., Adobe XD, Sketch, Figma), experience with wireframing and prototyping, understanding of user-centered design principles, and strong visual design skills."
    },
    {
        "job_name": "Product Manager",
        "job_description": "Oversee the development and lifecycle of a product from conception to launch. Required skills: Strong understanding of product management principles, experience with Agile methodologies, proficiency in project management tools (e.g., Jira, Trello), and strong communication and leadership abilities."
    },
    {
        "job_name": "System Administrator",
        "job_description": "Manage and maintain an organization's IT systems, ensuring they run smoothly and efficiently. Required skills: Proficiency in operating systems (e.g., Windows, Linux), experience with server administration, knowledge of network and system security practices, and strong troubleshooting skills."
    }
]


    # Step 4: Recommend jobs based on resume
    recommended_jobs = recommend_jobs(resume_text, jobs)
    print("Job Recommendations:")
    for job in recommended_jobs:
        print(f"{job['job_name']} -> {job['match']}% match")

if __name__ == "__main__":
    main()


Enter Path to resume: /content/1901841_RESUME.pdf
Resume Text:
ANUVA GOYAL   
D.O.B.: 1st October 2000  
Gender: Female    
OBJECTIVE  
 
Energetic, innovative engineering undergraduate, passionate about Machine Learning, NLP and Deep Learning for 
solving real -world problems, aiming to work in an organization providing great learning experience and growth 
opportunities for mutual benefit.  
 
EDUCATION  
 Qualification  Institute  CGPA  Year of Completion  
 
 B.Tech.  Dayalbagh Educational Institute,  CGPA 9.35  2023   
 
 (Electrical Engineering  (Till 4   
 
 Dayalbagh, Agra  Pursuing   
 Specialization in Computer Science)  semesters)   
    
 
 XII St. Clare’s Senior Secondary School, Agra  94%  2019   
 
 X St. Clare’s Senior Secondary School, Agra  CGPA 10  2017   
 
 INTERNSHIPS AND TRAININGS      
 
 
Internships:   
• Completed an internship at Genisup India Pvt. Ltd.  (July 2021 - Aug 2021) – NLP: Topic 
Modeling o Worked on proxy rotation and utilization  
o  Learnt Web 